In [1]:
import pandas as pd
from collections import defaultdict
import imaplib
import email
from email.header import decode_header
import os
from dotenv import load_dotenv
import re
from bs4 import BeautifulSoup
from imbox import Imbox
import imaplib
import io
from PIL import Image
import matplotlib.pyplot as plt
from pdf2image import convert_from_bytes

imaplib.IMAP4._encoding = 'utf-8'

In [2]:
# Initialize connection
load_dotenv()
imap_server = os.getenv("IMAP_SERVER")
email_address = os.getenv("EMAIL_ADDRESS")
password = os.getenv("EMAIL_PASSWORD")

mail = Imbox(imap_server, username=email_address, password=password, ssl=True, ssl_context=None, starttls=False)

In [3]:
filters = [
    {"sender": "noreply.tsekk@maxima.ee", "subject": "Sinu ostutšekk!"},
    {"sender": "noreply@rimibaltic.com", "subject": "Sinu ostutšekk"},
    {"sender": "estonia-food@bolt.eu", "subject": "Delivery from Bolt Food"}
]

In [4]:
filter_criteria = filters[1] # rimi

filter_sender = filter_criteria["sender"]
filter_subject = filter_criteria["subject"]

messages = mail.messages(sent_from=filter_sender)

In [5]:
for uid, message in messages:
    if message.subject == filter_subject:
        sent_from = message.sent_from
        sent_to = message.sent_to
        subject = message.subject
        headers = message.headers
        message_id = message.message_id
        date = message.date
        body = message.body
        attachments = message.attachments
        break

In [6]:
pdf_io = [a for a in attachments if a['content-type'] == 'application/pdf'][0]['content']

In [13]:
pdf_io.getvalue()

b'%PDF-1.4\n%\xe2\xe3\xcf\xd3\n1 0 obj\n<</Type/XObject/Subtype/Image/Width 346/Height 864/Filter/DCTDecode/ColorSpace/DeviceRGB/BitsPerComponent 8/Length 88705>>stream\n\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x03`\x01Z\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x8

In [15]:
io.BytesIO(pdf_io.getvalue())

In [16]:
pdf_io

In [19]:
with open("output.pdf", "wb") as f:
    f.write(pdf_io.getvalue())

In [22]:
from pdf2image import convert_from_path
pages = convert_from_path('output.pdf', 500)

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [20]:
import io
from pdf2image import convert_from_bytes
import pytesseract

images = convert_from_bytes(pdf_io.getvalue())

text = ""
for image in images:
    text += pytesseract.image_to_string(image)

print(text)


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [18]:
import io
from PyPDF2 import PdfReader


reader = PdfReader(pdf_io)

for page in reader.pages:
    print(page.extract_text())

In [7]:
# Convert PDF to a list of images
images = convert_from_bytes(pdf_io.getvalue())

# Display the first page as an example
for i, image in enumerate(images):
    image.show(title=f"Page {i + 1}")


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [112]:
# import requests, PyPDF2, io

# load_pdf = PyPDF2.PdfReader(pdf_io)

# # Extract text from each page
# pdf_text = ""
# for page_num in range(len(load_pdf.pages)):
#     pdf_text += load_pdf.pages[page_num].extract_text()

# print(pdf_text)

In [ ]:
def get_email_content(email_message):
    if email_message.is_multipart():
        for part in email_message.walk():
            print(part.get_content_type())
            if part.get_content_type() in ["text/plain", "text/html"]:
                return part.get_payload(decode=True).decode(errors='replace')
    else:
        return email_message.get_payload(decode=True).decode(errors='replace')


In [119]:
import io
from pdf2image import convert_from_bytes
from PIL import Image

# Assuming you have your PDF data in a BytesIO object
pdf_io = pdf_bytes  # Replace with your actual BytesIO object

# Extract raw bytes from the BytesIO object
pdf_bytes = pdf_io.getvalue()

# Convert PDF bytes to a list of images
images = convert_from_bytes(pdf_bytes)

# Display the first page as an example
for i, image in enumerate(images):
    image.show(title=f"Page {i + 1}")


AttributeError: 'bytes' object has no attribute 'getvalue'

In [18]:
num = message_numbers[0].split()[1]

_, msg_data = imap_connection.fetch(num, '(RFC822)')
email_body = msg_data[0][1]
email_message = email.message_from_bytes(email_body)

email_subject, encoding = decode_header(email_message["Subject"])[0]

if isinstance(email_subject, bytes):
    email_subject = email_subject.decode(encoding or "utf-8")

if email_subject == subject:
    email_sender = email_message["From"]
    email_date = email_message["Date"]
    
    content = get_email_content(email_message)
    
    filtered_emails[sender].append({
        "subject": email_subject,
        "sender": email_sender,
        "date": email_date,
        "content": content
    })

multipart/mixed
multipart/related


AttributeError: 'NoneType' object has no attribute 'decode'

In [12]:
content

'<html>\r\n\r\n<head>\r\n    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n</head>\r\n\r\n<table style="width:100%;font-family:Arial,sans-serif;font-weight:normal;line-height:0">\r\n    <tbody>\r\n    <tr>\r\n        <td style="background-color:white" align="center" valign="top">\r\n<table style="margin:0;padding:0;width:700px;line-height:normal" cellspacing="0" cellpadding="0" border="0" width="700">\r\n    <tbody><tr>\r\n        <td style="line-height:0;padding:60px 0 0 0;line-height:0" valign="top">\r\n            <img src="cid:bgImage" style="max-width:100%;line-height:0">\r\n        </td>\r\n    </tr>\r\n     </tbody>\r\n</table>\r\n\r\n<table style="margin:0;padding:0;width:770px" width="700" cellspacing="0" cellpadding="0" border="0">\r\n    <tbody>\r\n    <tr>\r\n        <td  style="background-color:white;padding:20px 0px 0 60px">\r\n             <h2 style="color:#363d40;font-size:24px;line-height:28px">Hea klient,</h2>\r\n             <br>\r\n       

In [ ]:
imap_connection.logout()